In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### load and clean recent_grads and grad_students datasets

In [2]:
# recent_grads
recent_grads = pd.read_csv('data/recent-grads.csv')

to_drop = ['Rank','Full_time_year_round', 'Unemployment_rate','ShareWomen', 'P25th','P75th',
           'Sample_size']
recent_grads.drop(to_drop, axis=1, inplace=True)
recent_grads['Major_code'] = recent_grads['Major_code'].apply(str)

recent_grads.dropna(subset = ['Total'], inplace=True)
recent_grads.isna().sum()
recent_grads.astype({'Total': 'int64', 'Men': 'int64', 'Women': 'int64'}).dtypes

recent_grads = recent_grads.rename(columns={'Median': 'Median_income'})

# Convert major name to title case
recent_grads['Major'] = recent_grads['Major'].str.title()

# Get short names of majors
recent_grads['Total'] = recent_grads['Total'].apply(np.int64)
recent_grads['Men'] = recent_grads['Men'].apply(np.int64)
recent_grads['Women'] = recent_grads['Women'].apply(np.int64)
major_list = pd.read_csv('data/major_short_names.csv')
recent_grads = pd.merge(major_list, recent_grads, how='right')
recent_grads.rename(columns={'Major':'Major_long', 'Major_short':'Major', 'Low_wage_jobs':'income: low-wage'}, inplace=True)
recent_grads['income: other'] = recent_grads['Employed'] - recent_grads['income: low-wage']

In [3]:
recent_grads.head()

,Major_category,Major_long,Major,Major_code,Total,Men,Women,Employed,Full_time,Part_time,Unemployed,Median_income,College_jobs,Non_college_jobs,income: low-wage,income: other
0,Engineering,Petroleum Engineering,Petroleum Eng.,2419,2339,2057,282,1976,1849,270,37,110000,1534,364,193,1783
1,Engineering,Mining And Mineral Engineering,Mining & Mineral Eng.,2416,756,679,77,640,556,170,85,75000,350,257,50,590
2,Engineering,Metallurgical Engineering,Metallurgical Eng.,2415,856,725,131,648,558,133,16,73000,456,176,0,648
3,Engineering,Naval Architecture And Marine Engineering,Naval Architecture & Marine Eng.,2417,1258,1123,135,758,1069,150,40,70000,529,102,0,758
4,Engineering,Chemical Engineering,Chemical Eng.,2405,32260,21239,11021,25694,23170,5180,1672,65000,18314,4440,972,24722


In [4]:
grad_students = pd.read_csv('data/grad-students.csv')

In [6]:
grad_students.columns

Index(['Major_code', 'Major', 'Major_category', 'Grad_total',
       'Grad_sample_size', 'Grad_employed', 'Grad_full_time_year_round',
       'Grad_unemployed', 'Grad_unemployment_rate', 'Grad_median', 'Grad_P25',
       'Grad_P75', 'Nongrad_total', 'Nongrad_employed',
       'Nongrad_full_time_year_round', 'Nongrad_unemployed',
       'Nongrad_unemployment_rate', 'Nongrad_median', 'Nongrad_P25',
       'Nongrad_P75', 'Grad_share', 'Grad_premium'],
      dtype='object')

In [7]:
to_drop = ['Major_code', 'Grad_sample_size', 'Grad_P25', 'Grad_P75',  'Nongrad_P25', 'Nongrad_P75', 'Grad_share', 'Grad_premium', 'Nongrad_full_time_year_round', 'Nongrad_median']
grad_students.drop(to_drop, axis=1, inplace=True)
grad_students.isna().sum()

Major                        0
Major_category               0
Grad_total                   0
Grad_employed                0
Grad_full_time_year_round    0
Grad_unemployed              0
Grad_unemployment_rate       0
Grad_median                  0
Nongrad_total                0
Nongrad_employed             0
Nongrad_unemployed           0
Nongrad_unemployment_rate    0
dtype: int64

In [8]:
grad_students.dtypes

Major                         object
Major_category                object
Grad_total                     int64
Grad_employed                  int64
Grad_full_time_year_round      int64
Grad_unemployed                int64
Grad_unemployment_rate       float64
Grad_median                  float64
Nongrad_total                  int64
Nongrad_employed               int64
Nongrad_unemployed             int64
Nongrad_unemployment_rate    float64
dtype: object

In [9]:
grad_students.rename(columns={'Grad_total':'Total', 'Grad_employed': 'Employed', 'Grad_full_time_year_round': 'Full_time',
                             'Grad_unemployed':'Unemployed', 'Nongrad_employed':'Non_college_jobs', 'Grad_median':'Median_income'}, inplace=True)
grad_students['Part_time'] = grad_students['Employed'] - grad_students['Full_time']
grad_students['Median_income'] = grad_students['Median_income'].apply(np.int64)
grad_students.head()

,Major,Major_category,Total,Employed,Full_time,Unemployed,Grad_unemployment_rate,Median_income,Nongrad_total,Non_college_jobs,Nongrad_unemployed,Nongrad_unemployment_rate,Part_time
0,CONSTRUCTION SERVICES,Industrial Arts & Consumer Services,9173,7098,6511,681,0.087543,75000,86062,73607,3928,0.050661,587
1,COMMERCIAL ART AND GRAPHIC DESIGN,Arts,53864,40492,29553,2482,0.057756,60000,461977,347166,25484,0.068386,10939
2,HOSPITALITY MANAGEMENT,Business,24417,18368,14784,1465,0.073867,65000,179335,145597,7409,0.048423,3584
3,COSMETOLOGY SERVICES AND CULINARY ARTS,Industrial Arts & Consumer Services,5411,3590,2701,316,0.080901,47000,37575,29738,1661,0.052900,889
4,COMMUNICATION TECHNOLOGIES,Computers & Mathematics,9109,7512,5622,466,0.058411,57000,53819,43163,3389,0.072800,1890


### pivot tables for visualizations

#### recent_grads

In [ ]:
recent_grads['Percent_employed'] = recent_grads['Employed'] / (recent_grads['Employed'] + recent_grads['Unemployed'])
recent_grads['Percent_college_jobs'] = recent_grads['College_jobs'] / (recent_grads['College_jobs'] + recent_grads['Non_college_jobs'])
recent_grads['Percent_non_college_jobs'] = recent_grads['Non_college_jobs'] / (recent_grads['College_jobs'] + recent_grads['Non_college_jobs'])
recent_grads['Total_income'] = recent_grads['Median_income'] * recent_grads['Full_time']
recent_grads['Percent_low_wage_jobs'] = recent_grads['income: low-wage'] / (recent_grads['College_jobs'] + recent_grads['Non_college_jobs'] + recent_grads['income: low-wage'])
recent_grads.head()

In [ ]:
df_cat = recent_grads.groupby('Major_category').agg({'Median_income': 'sum', 'Total': 'sum', 'Men': 'sum', 'Women': 'sum',
                    'Employed': 'sum', 'Unemployed': 'sum', 'Full_time': 'sum', 'College_jobs': 'sum',
                    'Non_college_jobs': 'sum', 'income: low-wage': 'sum'})
df_cat.reset_index(level=0, inplace=True)

# See note above about the approximation of median income by category
df_cat['Median_income'] = round(df_cat['Median_income'] / df_cat['Full_time'])
df_cat['Percent_employed'] = (df_cat['Employed'] / (df_cat['Employed'] + df_cat['Unemployed']))*100
df_cat['Percent_unemployed'] = (df_cat['Unemployed'] / (df_cat['Employed'] + df_cat['Unemployed']))*100
df_cat['Percent_men'] = (df_cat['Men'] / (df_cat['Men'] + df_cat['Women']))*100
df_cat['Percent_women'] = (df_cat['Women'] / (df_cat['Men'] + df_cat['Women']))*100
df_cat['Percent_college_jobs'] = (df_cat['College_jobs'] / (df_cat['College_jobs'] + df_cat['Non_college_jobs']))*100
df_cat['Percent_non_college_jobs'] = (df_cat['Non_college_jobs'] / (df_cat['College_jobs'] + df_cat['Non_college_jobs']))*100
df_cat['Percent_low_wage_jobs'] = df_cat['income: low-wage'] / (df_cat['College_jobs'] + df_cat['Non_college_jobs'] + df_cat['income: low-wage'])
df_cat.head()

In [ ]:
# Major categories sorted by median income - used to keep the same sorting as the other chart
sorted_categories = df_cat.sort_values(by=['Median_income'], ascending=False)['Major_category'].tolist()

In [ ]:
# Required for stacked bar chart: Data set with separate rows for "Percent_employed" and "Percent_unemployed"
df_cat_E = pd.melt(df_cat, id_vars=['Major_category', 'Median_income'], value_vars=['Percent_employed', 'Percent_unemployed'], 
        var_name='Status', value_name='Percentage')
df_cat_E.head()

In [ ]:
value_vars_C = ['Percent_college_jobs', 'Percent_non_college_jobs', 'Percent_low_wage_jobs']

df_cat_C = pd.melt(df_cat, id_vars=['Major_category'], value_vars=value_vars_C,
        var_name='Status', value_name='Percentage')
df_cat_C.head()

In [ ]:
RGC = pd.melt(recent_grads, id_vars=['Major', 'Major_category'], value_vars=value_vars_C,
        var_name='Status', value_name='Percentage')
RGC.head()

In [ ]:
recent_grads['unemployed'] = recent_grads['Unemployed'] / (recent_grads['Employed'] + recent_grads['Unemployed'])
recent_grads['employed'] = recent_grads['Employed'] / (recent_grads['Employed'] + recent_grads['Unemployed'])

# Split employed between college and non-college
recent_grads['employed (in-major)'] = recent_grads['employed']*recent_grads['College_jobs'] / (recent_grads['College_jobs'] + recent_grads['Non_college_jobs'])
recent_grads['employed (outside-major)'] = recent_grads['employed']*recent_grads['Non_college_jobs'] / (recent_grads['College_jobs'] + recent_grads['Non_college_jobs'])
recent_grads.head()

# Required for stacked bar chart: Data set with separate rows for 'Percent_unemployed', 'Percent_employed_college_jobs' and
# 'Percent_employed_non_college_jobs'

df_RGE = pd.melt(recent_grads, id_vars=['Major', 'Major_category'], 
              value_vars=['unemployed', 'employed (in-major)', 'employed (outside-major)'],
        var_name='Status', value_name='Percentage')
df_RGE.head()

In [ ]:
grad_students.head()

In [ ]:
# grad_students pivot tables

### visualizations

<ol>
<li>gendered violin plot by income</li>
<li>layered bar chart income vs major category, legend: undergraduates, graduates</li>
<li>dual-histograms for categories with large income change</li>
<li>total students vs income (popularity/income correlation)</li>
<li>stand-out individual majors (outliers within categories)</li>
<li>income vs employment type (display risk/reward tradeoff)</li>
<li>category/major gender charts (distribution for categories)</li>
</ul>

In [ ]:
# gendered violin plot by income

# layered bar chart income vs major category legend: undergraduates, graduates

# dual-histograms for categories with large income change

# total students vs income (popularity/income correlation)

# stand-out individual majors

# income vs employment type (display risk/reward tradeoff)

# category/major gender charts (distribution for categories)